## Import the required packages

In [1]:
pip install -q tflite-model-maker

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

ImportError: dlopen(/opt/anaconda3/lib/python3.9/site-packages/tensorflow_lite_support/python/task/audio/pybinds/_pywrap_audio_classifier.so, 0x0002): Library not loaded: /usr/local/opt/libusb/lib/libusb-1.0.0.dylib
  Referenced from: /opt/anaconda3/lib/python3.9/site-packages/tensorflow_lite_support/python/task/audio/pybinds/_pywrap_audio_classifier.so
  Reason: tried: '/usr/local/opt/libusb/lib/libusb-1.0.0.dylib' (no such file), '/usr/local/lib/libusb-1.0.0.dylib' (no such file), '/usr/lib/libusb-1.0.0.dylib' (no such file)

In [6]:
# Your labels map as a dictionary (zero is reserved):
label_map = {

     1 : "Pedestrian",
     2 : "People",
     3 : "Bicycle",
     4 : "Car",
     5 : "Van",
     6 : "Truck",
     7 : "Tricycle",
     8 : "Awning-tricycle",
     9 : "Bus",
     10 : "Motor",
     11 : "Others",
     12 : "None"
}
#file = open("Classeur1.txt","r")
#lignes = file.readlines()
#file.close()

#for ligne in lignes:
  #  element = ligne.split(";")
   # cle = int(element[0])
    #print(cle)
    #data = element[1]
    #print(data)
    #label_map[cle] = data
print (label_map)

# If it's NOT split yet, specify the path to all images and annotations
images_in = 'C:\\Users\\diazy\\Desktop\\Reentrainement\\VisDrone2019-DET-train\\images_new'
annotations_in = 'C:\\Users\\diazy\\Desktop\\Reentrainement\\VisDrone2019-DET-train\\annotations_new'
  

{1: 'Pedestrian', 2: 'People', 3: 'Bicycle', 4: 'Car', 5: 'Van', 6: 'Truck', 7: 'Tricycle', 8: 'Awning-tricycle', 9: 'Bus', 10: 'Motor', 11: 'Others', 12: 'None'}


In [7]:
#@markdown Be sure you run this cell. It's hiding the `split_dataset()` function used in the next code block.

import os
import random
import shutil

def split_dataset(images_path, annotations_path, val_split, test_split, out_path):
  """Splits a directory of sorted images/annotations into training, validation, and test sets.

  Args:
    images_path: Path to the directory with your images (JPGs).
    annotations_path: Path to a directory with your VOC XML annotation files,
      with filenames corresponding to image filenames. This may be the same path
      used for images_path.
    val_split: Fraction of data to reserve for validation (float between 0 and 1).
    test_split: Fraction of data to reserve for test (float between 0 and 1).
  Returns:
    The paths for the split images/annotations (train_dir, val_dir, test_dir)
  """
  _, dirs, _ = next(os.walk(images_path))

  train_dir = os.path.join(out_path, 'train')
  val_dir = os.path.join(out_path, 'validation')
  test_dir = os.path.join(out_path, 'test')

  IMAGES_TRAIN_DIR = os.path.join(train_dir, 'images_new')
  IMAGES_VAL_DIR = os.path.join(val_dir, 'images_new')
  IMAGES_TEST_DIR = os.path.join(test_dir, 'images_new')
  os.makedirs(IMAGES_TRAIN_DIR, exist_ok=True)
  os.makedirs(IMAGES_VAL_DIR, exist_ok=True)
  os.makedirs(IMAGES_TEST_DIR, exist_ok=True)

  ANNOT_TRAIN_DIR = os.path.join(train_dir, 'annotations_new')
  ANNOT_VAL_DIR = os.path.join(val_dir, 'annotations_new')
  ANNOT_TEST_DIR = os.path.join(test_dir, 'annotations_new')
  os.makedirs(ANNOT_TRAIN_DIR, exist_ok=True)
  os.makedirs(ANNOT_VAL_DIR, exist_ok=True)
  os.makedirs(ANNOT_TEST_DIR, exist_ok=True)

  # Get all filenames for this dir, filtered by filetype
  filenames = os.listdir(os.path.join(images_path))
  filenames = [os.path.join(images_path, f) for f in filenames if (f.endswith('.jpg'))]
  # Shuffle the files, deterministically
  filenames.sort()
  random.seed(42)
  random.shuffle(filenames)
  # Get exact number of images for validation and test; the rest is for training
  val_count = int(len(filenames) * val_split)
  test_count = int(len(filenames) * test_split)
  for i, file in enumerate(filenames):
    source_dir, filename = os.path.split(file)
    annot_file = os.path.join(annotations_path, filename.replace("jpg", "xml"))
    if i < val_count:
      shutil.copy(file, IMAGES_VAL_DIR)
      shutil.copy(annot_file, ANNOT_VAL_DIR)
    elif i < val_count + test_count:
      shutil.copy(file, IMAGES_TEST_DIR)
      shutil.copy(annot_file, ANNOT_TEST_DIR)
    else:
      shutil.copy(file, IMAGES_TRAIN_DIR)
      shutil.copy(annot_file, ANNOT_TRAIN_DIR)
  return (train_dir, val_dir, test_dir)

In [8]:
# We need to instantiate a separate DataLoader for each split dataset
train_dir, val_dir, test_dir = split_dataset(images_in, annotations_in,
                                                 val_split=0.2, test_split=0.2,
                                                 out_path='split-dataset')
train_data = object_detector.DataLoader.from_pascal_voc(
    os.path.join(train_dir, 'images_new'),
    os.path.join(train_dir, 'annotations_new'), label_map=label_map)
validation_data = object_detector.DataLoader.from_pascal_voc(
    os.path.join(val_dir, 'images_new'),
    os.path.join(val_dir, 'annotations_new'), label_map=label_map)
test_data = object_detector.DataLoader.from_pascal_voc(
    os.path.join(test_dir, 'images_new'),
    os.path.join(test_dir, 'annotations_new'), label_map=label_map)
    
print(f'train count: {len(train_data)}')
print(f'validation count: {len(validation_data)}')
print(f'test count: {len(test_data)}')

train count: 3464
validation count: 1154
test count: 1154


In [9]:
#spec = object_detector.EfficientDetLite0Spec(hparams{'max_instances_per_image': MAXOBJECTCOUNT})
spec = object_detector.EfficientDetLite3Spec(
  model_name='efficientdet-lite3', 
  uri='https://tfhub.dev/tensorflow/efficientdet/lite3/feature-vector/1', 
  hparams={'max_instances_per_image': 8000},
  strategy='gpus',
  model_dir = 'C:\\Users\\diazy\\Desktop\\Reentrainement\\new' )

In [10]:
import pycocotools


In [11]:
model = object_detector.create(train_data=train_data, 
                               model_spec=spec,  
                               epochs=10, 
                               batch_size=24,
                               train_whole_model=False)

OSError: SavedModel file does not exist at: C:\Users\diazy\AppData\Local\Temp\tfhub_modules\c683d6e0617380b32ffc04f38c79f57c18271787\{saved_model.pbtxt|saved_model.pb}

In [ ]:
#model = object_detector.create(train_data=train_data, 
                              # model_spec=spec, 
                               #validation_data=validation_data, 
                               #epochs=10, 
                               #batch_size=120,
                               
                               #train_whole_model=False)

In [11]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
model.evaluate(test_data)


21/21 [==============================] - 231s 9s/step



{'AP': 1.0411734e-06,
 'AP50': 4.4543203e-06,
 'AP75': 2.8513392e-07,
 'APs': 2.7567327e-07,
 'APm': 7.361273e-06,
 'APl': 3.1858395e-05,
 'ARmax1': 5.413599e-06,
 'ARmax10': 5.864732e-05,
 'ARmax100': 0.00037895193,
 'ARs': 0.00012726751,
 'ARm': 0.00048808317,
 'ARl': 0.0015245478,
 'AP_/Pedestrian': 0.0,
 'AP_/People': 0.0,
 'AP_/Bicycle': 0.0,
 'AP_/Car': 1.249408e-05,
 'AP_/Van': 0.0,
 'AP_/Truck': 0.0,
 'AP_/Tricycle': 0.0,
 'AP_/Awning-tricycle': 0.0,
 'AP_/Bus': 0.0,
 'AP_/Motor': 0.0,
 'AP_/Others': 0.0,
 'AP_/None': 0.0}

In [12]:
TFLITE_FILENAME = 'efficientdet-lite-new1.tflite'
LABELS_FILENAME = 'labels.txt'

In [13]:
model.export(export_dir='C:\\Users\\diazy\\Desktop\\Reentrainement', tflite_filename=TFLITE_FILENAME, label_filename=LABELS_FILENAME,
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

In [14]:
model.analyse()

AttributeError: 'ObjectDetector' object has no attribute 'analyse'

In [ ]:
model.evaluate_tflite(TFLITE_FILENAME, test_data)

In [ ]:
out_path='C:\\Users\\diazy\\Desktop\\Reentrainement\\split-dataset'
test_dir_small = os.path.join(out_path, 'test')
test_data_small = object_detector.DataLoader.from_pascal_voc(
    os.path.join(test_dir_small, 'images_small'),
    os.path.join(test_dir_small, 'annotations_small'), label_map=label_map)

In [ ]:
model.evaluate_tflite(TFLITE_FILENAME, test_data_small)

In [ ]:
import random

# If you're using a custom dataset, we take a random image from the test set:

images_path = test_images_dir if dataset_is_split else os.path.join(test_dir, "images_new")
filenames = os.listdir(os.path.join(images_path))
random_index = random.randint(0,len(filenames)-1)
INPUT_IMAGE = os.path.join(images_path, filenames[random_index])


To simplify our code, we'll use the [PyCoral API](https://coral.ai/docs/reference/py/):

In [ ]:
! python -m pip install --extra-index-url https://google-coral.github.io/py-repo/ pycoral

In [ ]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

#import tflite_runtime.interpreter as tflite 
from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file

def draw_objects(draw, objs, scale_factor, labels):
  """Draws the bounding box and label for each object."""
  COLORS = np.random.randint(0, 255, size=(len(labels), 3), dtype=np.uint8)
  for obj in objs:
    bbox = obj.bbox
    color = tuple(int(c) for c in COLORS[obj.id])
    draw.rectangle([(bbox.xmin * scale_factor, bbox.ymin * scale_factor),
                    (bbox.xmax * scale_factor, bbox.ymax * scale_factor)],
                   outline=color, width=3)
    font = ImageFont.truetype("LiberationSans-Regular.ttf", size=15)
    draw.text((bbox.xmin * scale_factor + 4, bbox.ymin * scale_factor + 4),
              '%s\n%.2f' % (labels.get(obj.id, obj.id), obj.score),
              fill=color, font=font)

# Load the TF Lite model
labels = read_label_file(LABELS_FILENAME)
interpreter = interpreter.Interpreter(TFLITE_FILENAME)
interpreter.allocate_tensors()

# Resize the image for input
image = Image.open(INPUT_IMAGE)
_, scale = common.set_resized_input(
    interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

# Run inference
interpreter.invoke()
objs = detect.get_objects(interpreter, score_threshold=0.4, image_scale=scale)

# Resize again to a reasonable size for display
display_width = 500
scale_factor = display_width / image.width
height_ratio = image.height / image.width
image = image.resize((display_width, int(display_width * height_ratio)))
draw_objects(ImageDraw.Draw(image), objs, scale_factor, labels)
image

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler

In [ ]:
NUMBER_OF_TPUS =  1

!edgetpu_compiler $TFLITE_FILENAME -d --num_segments=$NUMBER_OF_TPUS

In [ ]:
from google.colab import files

files.download(TFLITE_FILENAME)
files.download(TFLITE_FILENAME.replace('.tflite', '_edgetpu.tflite'))
files.download(LABELS_FILENAME)